## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-30-13-00-34 +0000,nyt,Second Victim of Dallas ICE Shooting Dies,https://www.nytimes.com/2025/09/30/us/dallas-i...
1,2025-09-30-13-00-00 +0000,missionlocal,The balloon,https://missionlocal.org/2025/09/sf-photos-the...
2,2025-09-30-13-00-00 +0000,wsj,How AI Helps a Logistics Giant Thrive During a...,https://www.wsj.com/articles/how-ai-helps-a-lo...
3,2025-09-30-12-56-53 +0000,nyt,Russia’s Military Budget Shrinks as War Costs ...,https://www.nytimes.com/2025/09/30/world/europ...
4,2025-09-30-12-56-17 +0000,nyt,South African Ambassador to France Is Found De...,https://www.nytimes.com/2025/09/30/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2291/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
30,trump,61
147,shutdown,21
146,government,20
104,new,18
32,gaza,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
135,2025-09-30-03-05-00 +0000,wsj,Top Democrats made their case to Trump for res...,https://www.wsj.com/politics/policy/government...,152
213,2025-09-29-21-14-29 +0000,nypost,Trump unveils 20-point Gaza cease-fire plan wi...,https://nypost.com/2025/09/29/us-news/trump-un...,143
113,2025-09-30-06-50-20 +0000,nypost,Trump mocks Chuck Schumer and Hakeem Jeffries ...,https://nypost.com/2025/09/30/us-news/trump-mo...,127
188,2025-09-29-22-29-55 +0000,nypost,Ex-UK PM Tony Blair to help run Gaza under Tru...,https://nypost.com/2025/09/29/us-news/ex-uk-pm...,122
8,2025-09-30-12-43-00 +0000,wsj,President Trump and Defense Secretary Pete Heg...,https://www.wsj.com/politics/national-security...,120


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
135,152,2025-09-30-03-05-00 +0000,wsj,Top Democrats made their case to Trump for res...,https://www.wsj.com/politics/policy/government...
213,82,2025-09-29-21-14-29 +0000,nypost,Trump unveils 20-point Gaza cease-fire plan wi...,https://nypost.com/2025/09/29/us-news/trump-un...
8,52,2025-09-30-12-43-00 +0000,wsj,President Trump and Defense Secretary Pete Heg...,https://www.wsj.com/politics/national-security...
238,50,2025-09-29-19-32-00 +0000,wsj,The clock is ticking down to an Oct. 1 deadlin...,https://www.wsj.com/politics/policy/government...
113,45,2025-09-30-06-50-20 +0000,nypost,Trump mocks Chuck Schumer and Hakeem Jeffries ...,https://nypost.com/2025/09/30/us-news/trump-mo...
19,42,2025-09-30-12-18-00 +0000,wsj,"Germany Jails Chinese Spy, Marking New Low in ...",https://www.wsj.com/world/europe/germany-jails...
246,40,2025-09-29-18-55-45 +0000,nypost,Frank founder Charlie Javice sobs in court as ...,https://nypost.com/2025/09/29/business/charlie...
165,39,2025-09-30-00-01-00 +0000,wsj,American Eagle Outfitters’ chief executive had...,https://www.wsj.com/business/retail/jay-schott...
6,37,2025-09-30-12-50-51 +0000,nypost,Second Dallas ICE shooting victim dies from in...,https://nypost.com/2025/09/30/us-news/second-d...
231,31,2025-09-29-20-30-47 +0000,nypost,Comcast names insider Michael Cavanagh to serv...,https://nypost.com/2025/09/29/media/comcast-na...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
